CSASL (and BCB and ESS, in particular) assembles, integrates, synthesizes, and produces data and information on individual species considered important for our primary mission of informing national conservation policy and resource management decisions. This work involves a number of individual activities that, together, form up one pillar of our overall biogeographic science practice and contribute to the formation of the National Biogeographic Map.

This notebook provides an overview of species data development, management, use, and distribution capabilities. It indicates the current state of play on foundational data management and areas we are working to improve.

In [1]:
import requests
import pysb
from IPython.display import display
from IPython.display import HTML

In [2]:
# In order to access private items in ScienceBase, we need to establish a connection using the pysb package. You will see summary output in this notebook from previous runs of the code when it is posted to GitHub, but we take care not to display particularly sensitive information from the in review items.
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: sbristol@usgs.gov
········


In [3]:
# Set up some parameters for this notebook
_gapHabitatMapCollectionItem = "527d0a83e4b0850ea0518326"
_gapSpeciesRangeCollectionItem = "5951527de4b062508e3b1e79"

### GAP Species Habitat Models/Maps
We are in the process of releasing the full slate of these products, many of which have been online and used in various ways for some time. Work has been done to refine the models and back-end processing, and all of this will be the subject of an upcoming data review and release. The final model/map products themselves are being assembled in ScienceBase under a [dedicated collection](https://www.sciencebase.gov/catalog/item/527d0a83e4b0850ea0518326) (still private pending review).

In [5]:
# Get the collection item for the GAP Habitat Maps so we can show summary information
gapHabitatMapCollection = sb.get_item(_gapHabitatMapCollectionItem,{'fields':'title,body,purpose,contacts'})

# Display a few summary fields for the habitat map collection
_shortReport = "<h3>"+gapHabitatMapCollection["title"]+"</h3>"
_shortReport = _shortReport+"<h4>Abstract</h4><p>"+gapHabitatMapCollection["body"]+"</p>"
_shortReport = _shortReport+"<h4>Purpose</h4><p>"+gapHabitatMapCollection["purpose"]+"</p>"
_shortReport = _shortReport+"<h4>Contacts</h4>"
for contact in gapHabitatMapCollection["contacts"]:
    _shortReport = _shortReport+"<div>"
    _shortReport = _shortReport+contact["name"]+" ("+contact["type"]+")"
    _shortReport = _shortReport+"</div>"
    
HTML(_shortReport)

In [15]:
# Report on the current number of GAP habitat map items in ScienceBase
habitatMapIDs = sb.get_child_ids(_gapHabitatMapCollectionItem)
_shortReport = "<div>Total number of GAP habitat maps in collection: <strong>"+str(len(habitatMapIDs))+"</strong></div>"

# Get a short list of habitat map items with a search and show a summary
for habitatMapID in habitatMapIDs[0:5]:
    habitatMapItem = sb.get_item(habitatMapID)
    # Display a few summary fields for the habitat map collection
    _shortReport = _shortReport+"<h3>"+habitatMapItem["title"]+"</h3>"
    _shortReport = _shortReport+"<h4>Abstract</h4><p>"+habitatMapItem["body"]+"</p>"
    _shortReport = _shortReport+"<h4>Identifiers</h4>"
    for identifier in habitatMapItem["identifiers"]:
        _shortReport = _shortReport+"<div>"
        _shortReport = _shortReport+identifier["type"]+" ("+identifier["key"]+")"
        _shortReport = _shortReport+"</div>"

HTML(_shortReport)


### GAP Species Ranges
While essentially a part of the GAP species modeling process, the species ranges are their own product with uses beyond the habitat modeling process. Ranges are built against a version of the hydrographic network, slightly modified from the original to meet GAP modeling needs. Range data are also housed in a [dedicated ScienceBase collection](https://www.sciencebase.gov/catalog/item/5951527de4b062508e3b1e79) (still private pending review).